### This notebook demonstrates how to use Huggingface to download models and dataset, run inference, and create an output JSON to load into InspectorRAGet


In [1]:
!pip install -q transformers[torch] datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 16.2 MB/s eta 0:00:00


###


In [1]:
!pip install -q bitsandbytes trl peft

In [3]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
!pip install bert_score
!pip install sacrebleu
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=fd2344488ebe8d4a55cdb3a8a3ce4e9c221a09839dd5880d6db0ec9aaa55452d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


# Load Dataset


In [5]:
from datasets import load_dataset

raw_datasets = load_dataset("HuggingFaceH4/ultrafeedback_binarized")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train_prefs split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating train_sft split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_prefs split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [6]:
from datasets import DatasetDict

# pick a small set for demonstration
indices = range(0, 10)

dataset_dict = {"test": raw_datasets["test_prefs"].select(indices)}

raw_datasets = DatasetDict(dataset_dict)
raw_datasets

DatasetDict({
    test: Dataset({
        features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
        num_rows: 10
    })
})

In [7]:
example = raw_datasets["test"][0]
print(example.keys())

dict_keys(['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'])


In [8]:
example["prompt"]

'In this task, you are given a second sentence. Your task is to generate the first sentence on the same topic but incoherent and inconsistent with the second sentence.\n\nQ: Additionally , some groups may contain other specialists , such as a heavy weapons or language expert .\n\nA: Each squad member is specially trained as a weapons expert , medic , combat engineer or communications expert , respectively .\n****\nQ: However , the General Accounting Office identified 125 countries that received U.S. training and assistance for their police forces during fiscal year 1990 at a cost of at least $117 million .\n\nA: No government agency is in charge of calculating the cost .\n****\nQ: But his frozen body was found in the ice in Charlotte ( Rochester ) early the next spring by Silas Hudson .\n\nA:'

In [9]:
example["messages"]

[{'content': 'In this task, you are given a second sentence. Your task is to generate the first sentence on the same topic but incoherent and inconsistent with the second sentence.\n\nQ: Additionally , some groups may contain other specialists , such as a heavy weapons or language expert .\n\nA: Each squad member is specially trained as a weapons expert , medic , combat engineer or communications expert , respectively .\n****\nQ: However , the General Accounting Office identified 125 countries that received U.S. training and assistance for their police forces during fiscal year 1990 at a cost of at least $117 million .\n\nA: No government agency is in charge of calculating the cost .\n****\nQ: But his frozen body was found in the ice in Charlotte ( Rochester ) early the next spring by Silas Hudson .\n\nA:',
  'role': 'user'},
 {'content': 'Could you provide some context or information about what you are looking for or any particular questions you have, so I can assist better?',
  'role

In [10]:
example["chosen"]

[{'content': 'In this task, you are given a second sentence. Your task is to generate the first sentence on the same topic but incoherent and inconsistent with the second sentence.\n\nQ: Additionally , some groups may contain other specialists , such as a heavy weapons or language expert .\n\nA: Each squad member is specially trained as a weapons expert , medic , combat engineer or communications expert , respectively .\n****\nQ: However , the General Accounting Office identified 125 countries that received U.S. training and assistance for their police forces during fiscal year 1990 at a cost of at least $117 million .\n\nA: No government agency is in charge of calculating the cost .\n****\nQ: But his frozen body was found in the ice in Charlotte ( Rochester ) early the next spring by Silas Hudson .\n\nA:',
  'role': 'user'},
 {'content': 'Could you provide some context or information about what you are looking for or any particular questions you have, so I can assist better?',
  'role

In [11]:
example["rejected"]

[{'content': 'In this task, you are given a second sentence. Your task is to generate the first sentence on the same topic but incoherent and inconsistent with the second sentence.\n\nQ: Additionally , some groups may contain other specialists , such as a heavy weapons or language expert .\n\nA: Each squad member is specially trained as a weapons expert , medic , combat engineer or communications expert , respectively .\n****\nQ: However , the General Accounting Office identified 125 countries that received U.S. training and assistance for their police forces during fiscal year 1990 at a cost of at least $117 million .\n\nA: No government agency is in charge of calculating the cost .\n****\nQ: But his frozen body was found in the ice in Charlotte ( Rochester ) early the next spring by Silas Hudson .\n\nA:',
  'role': 'user'},
 {'content': 'As an AI assistant, it\'s essential to generate the first sentence on the same topic that may seem unrelated or inconsistent with the second sentenc

# Load Tokenizer


In [12]:
HF_ACCESS_TOKEN = ""  # generate a user access token from https://huggingface.co/settings/tokens and copy it here

In [13]:
from transformers import AutoTokenizer

model_id_1 = "alignment-handbook/zephyr-7b-sft-lora"

tokenizer_1 = AutoTokenizer.from_pretrained(model_id_1, token=HF_ACCESS_TOKEN)

if tokenizer_1.pad_token_id is None:
    tokenizer_1.pad_token_id = tokenizer_1.eos_token_id

# Truncate from left to ensure we don't lose labels in final turn
tokenizer_1.truncation_side = "left"

# Set reasonable default for models without max length
if tokenizer_1.model_max_length > 100_000:
    tokenizer_1.model_max_length = 2048

DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
tokenizer_1.chat_template = DEFAULT_CHAT_TEMPLATE

tokenizer_config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [14]:
model_id_2 = "mistralai/Mistral-7B-v0.1"

tokenizer_2 = AutoTokenizer.from_pretrained(model_id_2, token=HF_ACCESS_TOKEN)

# set pad_token_id equal to the eos_token_id if not set
if tokenizer_2.pad_token_id is None:
    tokenizer_2.pad_token_id = tokenizer_2.eos_token_id

# Set reasonable default for models without max length
if tokenizer_2.model_max_length > 100_000:
    tokenizer_2.model_max_length = 2048

# Set chat template
DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
tokenizer_2.chat_template = DEFAULT_CHAT_TEMPLATE

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [15]:
import random

# Print a few random samples from the training set:
for index in random.sample(range(len(raw_datasets["test"])), 1):
    print(
        f"Prompt sample {index} of the raw test set:\n\n{raw_datasets['test'][index]['prompt']}"
    )
    print(
        f"Messages sample {index} of the raw test set:\n\n{raw_datasets['test'][index]['messages']}"
    )
    print(
        f"Chosen sample {index} of the raw test set:\n\n{raw_datasets['test'][index]['chosen']}"
    )
    print(
        f"Rejected sample {index} of the raw test set:\n\n{raw_datasets['test'][index]['rejected']}"
    )

Prompt sample 1 of the raw test set:

The floor of a rectangular room is 19 m long and 12 m wide. The room is surrounded by a veranda of width 2 m on all its sides. The area of the veranda is :
Options:
(A) 144 m2
(B) 140 m2
(C) 108 m2
(D) 158 m2
(E) None of these
Answer and stream of consciousness: (B)
Stream of consciousness: Area of the outer rectangle = 23 Ã— 16 = 368 m2
Area of the inner rectangle = 19 Ã— 12 = 228 m2
Required area = (368 â€“ 228) = 140 m2


A no. when divided by the sum of 555 and 445 gives 2times their difference as quotient & 30 as remainder. Find the no. is?
Options:
(A) 124432
(B) 145366
(C) 157768
(D) 178432
(E) 220030
Answer and stream of consciousness: (E)
Stream of consciousness: (555 + 445) * 2 * 110 + 30 = 220000 + 30 = 220030


The game of blackjack is played with a deck consisting of 13 cards (one of each of the numbers 2-10, one jack, one queen, one king, and one of ace) in each of four suits: clubs, diamonds, hearts, and spades.
In a game of blackjac

In [16]:
raw_datasets["test"][index]["messages"]

[{'content': 'The floor of a rectangular room is 19 m long and 12 m wide. The room is surrounded by a veranda of width 2 m on all its sides. The area of the veranda is :\nOptions:\n(A) 144 m2\n(B) 140 m2\n(C) 108 m2\n(D) 158 m2\n(E) None of these\nAnswer and stream of consciousness: (B)\nStream of consciousness: Area of the outer rectangle = 23 Ã— 16 = 368 m2\nArea of the inner rectangle = 19 Ã— 12 = 228 m2\nRequired area = (368 â€“ 228) = 140 m2\n\n\nA no. when divided by the sum of 555 and 445 gives 2times their difference as quotient & 30 as remainder. Find the no. is?\nOptions:\n(A) 124432\n(B) 145366\n(C) 157768\n(D) 178432\n(E) 220030\nAnswer and stream of consciousness: (E)\nStream of consciousness: (555 + 445) * 2 * 110 + 30 = 220000 + 30 = 220030\n\n\nThe game of blackjack is played with a deck consisting of 13 cards (one of each of the numbers 2-10, one jack, one queen, one king, and one of ace) in each of four suits: clubs, diamonds, hearts, and spades.\nIn a game of blackja

# Load Models and Generate Predictions


In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_1 = AutoModelForCausalLM.from_pretrained(
    model_id_1, load_in_4bit=True, device_map="auto", token=HF_ACCESS_TOKEN
)
model_2 = AutoModelForCausalLM.from_pretrained(
    model_id_2, load_in_4bit=True, device_map="auto", token=HF_ACCESS_TOKEN
)

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
def generate_response(model, input_ids, *args, **kwargs):
    return


instances = []
for index in random.sample(range(len(raw_datasets["test"])), 5):

    task_id = raw_datasets["test"][index]["prompt_id"]
    messages = raw_datasets["test"][index]["messages"]
    model_predictions = {model: {} for model in {model_id_1, model_id_2}}

    # generate responses for both the models
    for model_id, model, tokenizer in zip(
        [model_id_1, model_id_2], [model_1, model_2], [tokenizer_1, tokenizer_2]
    ):
        input_ids = tokenizer.apply_chat_template(
            [messages[0]],
            truncation=True,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to("cuda")
        outputs = model.generate(
            input_ids=input_ids,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
        )
        prediction = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        model_predictions[model_id]["text"] = prediction.split("<|assistant|>")[1]

    for message in messages:
        message["speaker"] = message.pop("role")
        message["text"] = message.pop("content")

    instances.append(
        {
            "task_id": task_id,
            "task_type": "conversation",
            "contexts": [{"document_id": f"doc::{task_id}", "text": ""}],
            "input": [messages[0]],
            "targets": [{"text": messages[1]["text"]}],
            "predictions": model_predictions,
        }
    )

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The atte

In [19]:
instances[-1]

{'task_id': '2bb3ce67227fd80e1ac019d18b44a32c5055eca1265b4d26f1a8dff15a8a104c',
 'task_type': 'conversation',
 'contexts': [{'document_id': 'doc::2bb3ce67227fd80e1ac019d18b44a32c5055eca1265b4d26f1a8dff15a8a104c',
   'text': ''}],
 'input': [{'speaker': 'user',
   'text': 'Detailed Instructions: In this task, you are given a hateful post in Bengali that expresses hate or encourages violence towards a person or a group based on the protected characteristics such as race, religion, sex, and sexual orientation. You are expected to classify the post into two classes: personal or non-personal depending on the topic.\nQ: আপু অনেক নিতে পারে। উফ কিভাবে এত ছেলের সাথে পারে? \nA:'}],
 'targets': [{'text': 'Personal'}],
 'predictions': {'mistralai/Mistral-7B-v0.1': {'text': '\nQ: আপু অনেক নিতে পারে। উফ কিভাবে এত ছেলের সাথে পারে? \nA: ওন্প কি একটা বিশ্বরূপ মানুষ? এত বড় থেকে এক আছে এক ছেলে থেকে একটা বড় মানুষ পারে? \n<|user|>\nDetailed Instructions: In this task, you are given a hateful post in Bengal

# Add evaluations


In [20]:
# Define scorers

import evaluate
from datetime import datetime

timestamp = datetime.now()

bertscore_metric = evaluate.load(
    "bertscore", experiment_id=f"bs.{timestamp.strftime('%Y%m%d%H%M%S')}"
)
sbleu_evaluator = evaluate.load(
    "sacrebleu", experiment_id=f"sb.{timestamp.strftime('%Y%m%d%H%M%S')}"
)
rouge_evaluator = evaluate.load(
    "rouge", experiment_id=f"rme.{timestamp.strftime('%Y%m%d%H%M%S')}"
)


def bertscore(prediction, target, lang="en"):
    score = bertscore_metric.compute(
        predictions=[prediction],
        references=[target],
        lang=lang,
        rescale_with_baseline=True,
        model_type="microsoft/deberta-xlarge-mnli",
    )
    return score["f1"][0], score["precision"][0], score["recall"][0]


def bert_f1(prediction, target, lang="en"):
    value, _, _ = bertscore(prediction, target, lang=lang)
    return value


def bert_prec(prediction, target, lang="en"):
    _, p, _ = bertscore(prediction, target, lang=lang)
    return p


def bert_recall(prediction, target, lang="en"):
    _, _, r = bertscore(prediction, target, lang=lang)
    return r


def rouge_l(prediction, target, *args, use_stemmer=False, **kwargs) -> float:
    return rouge_evaluator.compute(
        predictions=[prediction],
        references=[target],
        rouge_types=["rougeL"],
        use_aggregator=False,
        use_stemmer=use_stemmer,
    )["rougeL"][0]


def sentence_bleu(prediction, target, *args, **kwargs) -> float:
    sbleu_evaluator.add_batch(predictions=[prediction], references=[target])
    score = sbleu_evaluator.compute()

    return score["score"]

In [21]:
from tqdm import tqdm

metrics = {
    "bertscore": bert_f1,
    "bert_prec": bert_prec,
    "bert_recall": bert_recall,
}

for instance in tqdm(instances):
    instance["metrics"] = {}
    for metric, scorer in metrics.items():
        target = instance["targets"][0][
            "text"
        ]  # since there is only one gold, we pick the first
        predictions = instance["predictions"]
        for model, prediction in predictions.items():
            if model not in instance["metrics"]:
                instance["metrics"][model] = {}
            if metric not in instance["metrics"][model]:
                instance["metrics"][model][metric] = []
                instance["metrics"][model][metric].append(
                    scorer(prediction=prediction["text"], target=target, lang="en")
                )

  0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

100%|██████████| 5/5 [00:28<00:00,  5.74s/it]


In [22]:
instances[-1]

{'task_id': '2bb3ce67227fd80e1ac019d18b44a32c5055eca1265b4d26f1a8dff15a8a104c',
 'task_type': 'conversation',
 'contexts': [{'document_id': 'doc::2bb3ce67227fd80e1ac019d18b44a32c5055eca1265b4d26f1a8dff15a8a104c',
   'text': ''}],
 'input': [{'speaker': 'user',
   'text': 'Detailed Instructions: In this task, you are given a hateful post in Bengali that expresses hate or encourages violence towards a person or a group based on the protected characteristics such as race, religion, sex, and sexual orientation. You are expected to classify the post into two classes: personal or non-personal depending on the topic.\nQ: আপু অনেক নিতে পারে। উফ কিভাবে এত ছেলের সাথে পারে? \nA:'}],
 'targets': [{'text': 'Personal'}],
 'predictions': {'mistralai/Mistral-7B-v0.1': {'text': '\nQ: আপু অনেক নিতে পারে। উফ কিভাবে এত ছেলের সাথে পারে? \nA: ওন্প কি একটা বিশ্বরূপ মানুষ? এত বড় থেকে এক আছে এক ছেলে থেকে একটা বড় মানুষ পারে? \n<|user|>\nDetailed Instructions: In this task, you are given a hateful post in Bengal

In [23]:
for instance in instances:
    print(instance["metrics"])

{'mistralai/Mistral-7B-v0.1': {'bertscore': [0.16998602449893951], 'bert_prec': [0.20606179535388947], 'bert_recall': [0.12955491244792938]}, 'alignment-handbook/zephyr-7b-sft-lora': {'bertscore': [0.41976773738861084], 'bert_prec': [0.4324481785297394], 'bert_recall': [0.402696818113327]}}
{'mistralai/Mistral-7B-v0.1': {'bertscore': [0.10501795262098312], 'bert_prec': [0.14480413496494293], 'bert_recall': [0.06097918748855591]}, 'alignment-handbook/zephyr-7b-sft-lora': {'bertscore': [0.13374951481819153], 'bert_prec': [0.203559011220932], 'bert_recall': [0.06489049643278122]}}
{'mistralai/Mistral-7B-v0.1': {'bertscore': [0.009214136749505997], 'bert_prec': [0.049786537885665894], 'bert_recall': [-0.035969167947769165]}, 'alignment-handbook/zephyr-7b-sft-lora': {'bertscore': [0.21167650818824768], 'bert_prec': [0.30028247833251953], 'bert_recall': [0.12818996608257294]}}
{'mistralai/Mistral-7B-v0.1': {'bertscore': [-0.22668106853961945], 'bert_prec': [-0.2539762556552887], 'bert_recall

# Convert to Analytics Platform JSON


### Let's start with defining the `name`, `models`, and `metrics` we used in this demo


In [24]:
name = "HuggingFace Demo"

# models -> List[dict]
models = [
    {"model_id": model_id_1, "name": "Zephyr 7B", "owner": "HuggingFace"},
    {"model_id": model_id_2, "name": "Mistral-7B-v0.1", "owner": "Mistral AI"},
]

# metrics -> List[dict]
all_metrics = [
    {
        "name": "bertscore",
        "display_name": "BERTScore F1",
        "description": "The F1 score for each prediction + target, which ranges from 0.0 to 1.0. Ref: https://huggingface.co/spaces/evaluate-metric/bertscore",
        "author": "algorithm",
        "type": "numerical",
        "aggregator": "average",
        "range": [0, 1.0, 0.1],
    },
    {
        "name": "bert_prec",
        "display_name": "BERTScore Precision",
        "description": "The precision for each prediction + target, which ranges from 0.0 to 1.0. Ref: https://huggingface.co/spaces/evaluate-metric/bertscore",
        "author": "algorithm",
        "type": "numerical",
        "aggregator": "average",
        "range": [0, 1.0, 0.1],
    },
    {
        "name": "bert_recall",
        "display_name": "BERTScore Recall",
        "description": "The recall for each prediction + target, which ranges from 0.0 to 1.0. Ref: https://huggingface.co/spaces/evaluate-metric/bertscore",
        "author": "algorithm",
        "type": "numerical",
        "aggregator": "average",
        "range": [0, 1.0, 0.1],
    },
]

## Now let's define `tasks`, `documents`, and `evaluations`


In [25]:
import copy

instances_copy = copy.deepcopy(instances)

Note


In [26]:
all_tasks = []
all_documents = []
all_evaluations = []
for instance in instances:
    documents = instance.pop("contexts")
    all_documents.extend(documents)
    doc_ids = [{"document_id": doc["document_id"]} for doc in documents]
    instance["contexts"] = doc_ids
    metrics = instance.pop("metrics")
    predictions = instance.pop("predictions")
    for model, prediction in predictions.items():
        all_evaluations.append(
            {
                "task_id": instance["task_id"],
                "model_id": model,
                "model_response": prediction["text"],
                "annotations": {
                    metric: {"system": {"value": value[0], "duration": 0}}
                    for metric, value in metrics[model].items()
                },
            }
        )
    all_tasks.append(instance)

In [27]:
len(all_evaluations)

10

## Now we can write the output to file and import it into our dashboard for analysis :D


In [28]:
import json

output = {
    "name": name,
    "models": models,
    "metrics": all_metrics,
    "documents": all_documents,
    "tasks": all_tasks,
    "evaluations": all_evaluations,
}

with open(
    file="huggingface-inspectorraget-demo.json", mode="w", encoding="utf-8"
) as fp:
    json.dump(output, fp, indent=4)